# CIA Factbook

In this project we will query CIA Factbook, a database containing general information on the countries of the world.

Here are the descriptions for some of the columns:

`name` — the name of the country.
`area` — the country's total area (both land and water).
`area_land` — the country's land area in square kilometers.
`area_water` — the country's waterarea in square kilometers.
`population` — the country's population.
`population_growth` — the country's population growth as a percentage.
`birth_rate` — the country's birth rate, or the number of births per year per 1,000 people.
`death_rate` — the country's death rate, or the number of death per year per 1,000 people.

We will first install the necessary tools.

In [1]:
!conda install -yc conda-forge ipython-sql


Solving environment: done

## Package Plan ##

  environment location: /opt/anaconda3

  added / updated specs:
    - ipython-sql


The following NEW packages will be INSTALLED:

  python_abi         conda-forge/osx-64::python_abi-3.8-2_cp38

The following packages will be UPDATED:

  conda              pkgs/main::conda-4.10.3-py38hecd8cb5_0 --> conda-forge::conda-4.10.3-py38h50d1736_3


Preparing transaction: done
Verifying transaction: done
Executing transaction: done


In [2]:
%%capture
%load_ext sql
%sql sqlite:///factbook.db

In [3]:
%%sql
SELECT *
  FROM sqlite_master
 WHERE type='table';

 * sqlite:///factbook.db
Done.


type,name,tbl_name,rootpage,sql
table,sqlite_sequence,sqlite_sequence,3,"CREATE TABLE sqlite_sequence(name,seq)"
table,facts,facts,47,"CREATE TABLE ""facts"" (""id"" INTEGER PRIMARY KEY AUTOINCREMENT NOT NULL, ""code"" varchar(255) NOT NULL, ""name"" varchar(255) NOT NULL, ""area"" integer, ""area_land"" integer, ""area_water"" integer, ""population"" integer, ""population_growth"" float, ""birth_rate"" float, ""death_rate"" float, ""migration_rate"" float)"


Now let's explore the data a little.

In [4]:
%%sql
SELECT *
  FROM facts
 LIMIT 5;

 * sqlite:///factbook.db
Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
1,af,Afghanistan,652230,652230,0,32564342,2.32,38.57,13.89,1.51
2,al,Albania,28748,27398,1350,3029278,0.3,12.92,6.58,3.3
3,ag,Algeria,2381741,2381741,0,39542166,1.84,23.67,4.31,0.92
4,an,Andorra,468,468,0,85580,0.12,8.13,6.96,0.0
5,ao,Angola,1246700,1246700,0,19625353,2.78,38.78,11.49,0.46


Now we will colculate some summary statistics and look for any outlier countries

In [6]:
%%sql
SELECT MIN(population), MAX(population), MIN(population_growth), MAX(population_growth)
  FROM facts;

 * sqlite:///factbook.db
Done.


MIN(population),MAX(population),MIN(population_growth),MAX(population_growth)
0,7256490011,0.0,4.02


In [10]:
%%sql
SELECT name
  FROM facts
 WHERE population = (SELECT MIN(population)
                       FROM facts
                    );

 * sqlite:///factbook.db
Done.


name
Antarctica


In [9]:
%%sql
SELECT name
  FROM facts
 WHERE population_growth = (SELECT MIN(population_growth)
                              FROM facts
                           );

 * sqlite:///factbook.db
Done.


name
Holy See (Vatican City)
Cocos (Keeling) Islands
Greenland
Pitcairn Islands


In [12]:
%%sql
SELECT name
  FROM facts
 WHERE population = (SELECT MAX(population)
                              FROM facts
                    );

 * sqlite:///factbook.db
Done.


name
World


So, the country with population of 0 is Antarctica, which makes sense, same as the World with over 7 billion people. These values may skew our analysis though, so we should drop them and focus on actual countries.

In [13]:
%%sql
SELECT MAX(population), MIN(population), MAX(population_growth), MIN(population_growth)
  FROM facts
 WHERE name != 'World' AND name != 'Antarctica';

 * sqlite:///factbook.db
Done.


MAX(population),MIN(population),MAX(population_growth),MIN(population_growth)
1367485388,48,4.02,0.0


Now that we found the outliers, we can compute the average population and area of world countries.

In [14]:
%%sql
SELECT AVG(population), AVG(area)
  FROM facts
 WHERE name != 'World' AND name != 'Antarctica';

 * sqlite:///factbook.db
Done.


AVG(population),AVG(area)
32377011.0125,555093.546184739


Now, I will build on the query I wrote above to find countries that are densely populated. We'll identify countries that have the following:

- Above-average values for population and
- Below-average values for area.

In [16]:
%%sql
SELECT name, population, area
  FROM facts
 WHERE population > (SELECT AVG(population)
                      FROM facts
                     WHERE name != 'World' AND name != 'Antarctica'
                    ) AND
       area < (SELECT AVG(area)
                 FROM facts
                WHERE name != 'World' AND name != 'Antarctica'
              );

 * sqlite:///factbook.db
Done.


name,population,area
Bangladesh,168957745,148460
Germany,80854408,357022
Iraq,37056169,438317
Italy,61855120,301340
Japan,126919659,377915
"Korea, South",49115196,99720
Morocco,33322699,446550
Philippines,100998376,300000
Poland,38562189,312685
Spain,48146134,505370


Which country has the most people? 

In [18]:
%%sql 
SELECT name, population
  FROM facts
 WHERE population = (SELECT MAX(population)
                           FROM facts
                          WHERE name != 'World' AND name != 'Antarctica'
                        );

 * sqlite:///factbook.db
Done.


name,population
China,1367485388


Which country has the highest growth rate?

In [20]:
%%sql 
SELECT name, population_growth
  FROM facts
 WHERE population_growth = (SELECT MAX(population_growth)
                           FROM facts
                    );

 * sqlite:///factbook.db
Done.


name,population_growth
South Sudan,4.02


Which countries have more water than land?


In [21]:
%%sql 
SELECT name, area_water, area_land
  FROM facts
 WHERE area_water > area_land;

 * sqlite:///factbook.db
Done.


name,area_water,area_land
British Indian Ocean Territory,54340,60
Virgin Islands,1564,346


Which countries have the highest ratios of water to land? I will show top ten maritime countries.

In [25]:
%%sql 
SELECT name, (CAST(area_water AS Float)/area_land) AS water_ratio
  FROM facts
 ORDER BY water_ratio DESC
 LIMIT 10;

 * sqlite:///factbook.db
Done.


name,water_ratio
British Indian Ocean Territory,905.6666666666666
Virgin Islands,4.520231213872832
Puerto Rico,0.5547914317925592
"Bahamas, The",0.3866133866133866
Guinea-Bissau,0.2846728307254623
Malawi,0.25939625850340137
Netherlands,0.22571032366565366
Uganda,0.22292237442922375
Eritrea,0.16435643564356436
Liberia,0.15623961794019933


Which countries will add the most people to their populations next year?

In [29]:
%%sql 
SELECT name, CAST(birth_rate AS INT) - CAST (death_rate AS INT) AS new_people
  FROM facts
 ORDER BY new_people DESC
 LIMIT 20;

 * sqlite:///factbook.db
Done.


name,new_people
Burundi,33
Malawi,33
Niger,33
Uganda,33
Mali,32
Burkina Faso,31
Zambia,30
Ethiopia,29
Benin,28
Iraq,28


Which countries have a higher death rate than birth rate?

In [30]:
%%sql 
SELECT name, birth_rate, death_rate 
  FROM facts
 WHERE death_rate > birth_rate;

 * sqlite:///factbook.db
Done.


name,birth_rate,death_rate
Austria,9.41,9.42
Belarus,10.7,13.36
Bosnia and Herzegovina,8.87,9.75
Bulgaria,8.92,14.44
Croatia,9.45,12.18
Czech Republic,9.63,10.34
Estonia,10.51,12.4
Germany,8.47,11.42
Greece,8.66,11.09
Hungary,9.16,12.73


Which countries have the highest population/area ratio?

In [34]:
%%sql 
SELECT name, population, area, population/area AS population_area_ratio
  FROM facts
 ORDER BY population_area_ratio DESC
 LIMIT 20;

 * sqlite:///factbook.db
Done.


name,population,area,population_area_ratio
Macau,592731,28,21168
Monaco,30535,2,15267
Singapore,5674472,697,8141
Hong Kong,7141106,1108,6445
Gaza Strip,1869055,360,5191
Gibraltar,29258,6,4876
Bahrain,1346613,760,1771
Maldives,393253,298,1319
Malta,413965,316,1310
Bermuda,70196,54,1299


Counterintuitively, we have a different list of countries than when in `cell 16`.